# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.209/12507/1 Dashboard: http://10.20.0.209:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1035,Edith,-0.81738715977504,-0.2954281783368695
2000-01-01 00:00:01,972,Xavier,-0.32454347096255365,0.197756533552496
2000-01-01 00:00:02,993,Ray,-0.838285790230207,-0.7480155493079506
2000-01-01 00:00:03,997,Tim,0.07912484338774717,-0.658145798637596
2000-01-01 00:00:04,1012,Ursula,-0.20333954405656973,-0.4330319312543296
2000-01-01 00:00:05,972,Quinn,-0.8471625391425903,0.18864741873082003
2000-01-01 00:00:06,992,Laura,-0.28333987575609854,0.5973140740050189
2000-01-01 00:00:07,1049,Michael,0.8939627387893583,-0.3756596501182794
2000-01-01 00:00:08,1041,Wendy,0.45384589839322276,-0.29147645321431326


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1012,Laura,-0.9026355112599023,0.6371789293317187
2000-01-30 00:00:01,1039,Bob,-0.6987385667007462,0.5956342849052767
2000-01-30 00:00:02,993,Alice,0.6888193176054156,-0.14781709854631697
2000-01-30 00:00:03,993,Laura,0.7972356566611647,0.45118197880236655
2000-01-30 00:00:04,986,Oliver,-0.7269268419811212,0.7945865894761777
2000-01-30 00:00:05,978,Ray,-0.8020044248388054,-0.04010751500673804
2000-01-30 00:00:06,1037,Kevin,-0.9931028922792529,0.6148117758959335
2000-01-30 00:00:07,1015,Zelda,-0.018976684961328427,-0.07401804599248507
2000-01-30 00:00:08,932,Jerry,0.025145554128741976,0.9725005672800533


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1035,Edith,-0.817387,-0.295428
1,2000-01-01 00:00:01,972,Xavier,-0.324543,0.197757
2,2000-01-01 00:00:02,993,Ray,-0.838286,-0.748016
3,2000-01-01 00:00:03,997,Tim,0.079125,-0.658146
4,2000-01-01 00:00:04,1012,Ursula,-0.203340,-0.433032


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1035,Edith,-0.817387,-0.295428
1,2000-01-01 00:00:01,972,Xavier,-0.324543,0.197757
2,2000-01-01 00:00:02,993,Ray,-0.838286,-0.748016
3,2000-01-01 00:00:03,997,Tim,0.079125,-0.658146
4,2000-01-01 00:00:04,1012,Ursula,-0.203340,-0.433032


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.37 s, sys: 677 ms, total: 5.05 s
Wall time: 3.26 s


name
Alice       0.000667
Bob        -0.001327
Charlie     0.000652
Dan         0.000848
Edith      -0.000455
Frank       0.001689
George      0.000792
Hannah      0.000614
Ingrid      0.001566
Jerry      -0.000740
Kevin       0.000989
Laura       0.000594
Michael    -0.001314
Norbert     0.002493
Oliver     -0.000613
Patricia    0.003627
Quinn       0.000646
Ray         0.003386
Sarah       0.000985
Tim         0.000045
Ursula     -0.000490
Victor      0.002584
Wendy       0.002587
Xavier      0.001396
Yvonne     -0.001098
Zelda       0.002624
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.38 s, sys: 160 ms, total: 1.54 s
Wall time: 1.03 s


name
Alice       0.000667
Bob        -0.001327
Charlie     0.000652
Dan         0.000848
Edith      -0.000455
Frank       0.001689
George      0.000792
Hannah      0.000614
Ingrid      0.001566
Jerry      -0.000740
Kevin       0.000989
Laura       0.000594
Michael    -0.001314
Norbert     0.002493
Oliver     -0.000613
Patricia    0.003627
Quinn       0.000646
Ray         0.003386
Sarah       0.000985
Tim         0.000045
Ursula     -0.000490
Victor      0.002584
Wendy       0.002587
Xavier      0.001396
Yvonne     -0.001098
Zelda       0.002624
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.25 s, sys: 60.6 ms, total: 1.31 s
Wall time: 899 ms


name
Alice       0.000667
Bob        -0.001327
Charlie     0.000652
Dan         0.000848
Edith      -0.000455
Frank       0.001689
George      0.000792
Hannah      0.000614
Ingrid      0.001566
Jerry      -0.000740
Kevin       0.000989
Laura       0.000594
Michael    -0.001314
Norbert     0.002493
Oliver     -0.000613
Patricia    0.003627
Quinn       0.000646
Ray         0.003386
Sarah       0.000985
Tim         0.000045
Ursula     -0.000490
Victor      0.002584
Wendy       0.002587
Xavier      0.001396
Yvonne     -0.001098
Zelda       0.002624
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html